In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping



import unicodedata
import re
from collections import Counter



<style>
.center {
  display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
}
</style>

<img src="https://i.ibb.co/NbtsTwH/patito-goyito.png" alt="Portada-T5-p1" border="0" class="center">

### 1.3. Preguntas a responder

* Autoencoders: investigue sobre los modelos autoencoders, mencione a grandes rasgos cómo es, su estructura y responda: ¿Por qué son útiles para comprimir y descomprimir datos? (extensión máxima 7 líneas)
* Tokenización: investigue acerca de la tokenización de los textos. Describa brevemente el concepto y plantee un esquema de tokenización para sus datos. En este sentido, justifique el esquema que utilizaría, ya sea por palabras o por caracteres. (extensión máxima 7 líneas)
* Funciones de pérdida: investigue sobre las funciones de pérdida y en particular sobre: Cross
Entropy Loss. Explica cómo funciona y reflexiona sobre su utilidad en este contexto. (Extensión máxima 9 líneas).
* Dropout: discuta sobre la utilidad de utilizar dropout en este contexto. Indique además en cuales capas de un autoencoder utilizaría dropout y en cuáles no. (Extensión máxima 10 líneas).

### 1.5.    Análisis del Modelo de Aprendizaje Profundo

Hiperparámetros: Explique la elección de hiperparámetros que realizó; entiéndase por hiperparámetros elecciones como épocas de entrenamiento, longitud máxima de las frases, numero de capas, tamaño˜ de las capas, etc.
Resultados: Presente una muestra de frases originales, vector de compresión Z y descompresiones hechas por el modelo, determine si su modelo tiende a fallar más en frases cortas o largas, y explique por qué razón podría ocurrir esto.

Exactitud: Determine la cantidad de aciertos que tiene su modelo para predecir frases por medio de la cantidad de tokens en la posición correcta que el modelo entregó, obtenga al menos un 50% de acorace a nivel de token por posición. También calcule la cantidad de aciertos por frecuencia de palabras entre la frase original y la frase generada, sin importar el lugar en el que esté un token.

Comente sobre sus resultados y qué pueden indicar los valores obtenidos sobre cómo el modelo predice las palabras. (extensión máxima 3-4 líneas)
Errores de palabras: Comente sobre cuáles son las palabras que más errores presenta su modelo, y comente a qué podría deberse esto (frecuencia, ambigüedad, falta de datos, etc.). (extensión máxima 5 líneas)

Compresor: Comente si este tipo de modelos es bueno para realizar compresión de texto. Con base en lo que investigo y determinó con sus resultados, decida si serviría o no para este propósito, ya sea con mejores ajustes, mayor capacidad de cómputo, entrenamiento, más memoria, etc. (extensión máxima 5-7 líneas)

Seq2Seq: Investigue sobre las redes Seq2Seq, mencione diferencias y similitudes sobre el modelo que usted desarrolló y este tipo de redes. (extensión máxima 5-6 líneas)




El dataset que ocupe fue recuperado de Kaggle llamado [Quotes- 500k](https://www.kaggle.com/datasets/manann/quotes-500k/data) por Manan.

Este tiene 500.000 frases por famosos.

In [2]:
quotes = pd.read_csv("quotes.csv")

quotes.drop(columns=["author", "category"], inplace=True)

quotes.head()

,quote
0,"I'm selfish, impatient and a little insecure. ..."
1,You've gotta dance like there's nobody watchin...
2,You know you're in love when you can't fall as...
3,A friend is someone who knows all about you an...
4,Darkness cannot drive out darkness: only light...


##### Preprocesamiento

In [3]:
# Limpiemos los textos

def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()

    # Eliminar acentos
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)

    # Eliminar espacios extra
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [4]:
# Función para ver el numero de nulos y duplicados

def check_nulls_and_duplicates(df):
    nulls = df.isnull().sum()
    duplicates = df.duplicated().sum()

    return nulls, duplicates


In [5]:
quotes_cleaned = quotes.copy()

# Verificar nulos y duplicados
print("🔍 Buscando nulos y duplicados 🔍\n")
nulls, duplicates = check_nulls_and_duplicates(quotes_cleaned)
print(f"Hay {nulls.sum()} nulos en total\n")
print(f"Hay {duplicates} filas duplicadas\n")

🔍 Buscando nulos y duplicados 🔍

Hay 1 nulos en total

Hay 5919 filas duplicadas



In [6]:
# Vemos que hay 5919 filas duplicadas, las eliminamos
quotes_cleaned.drop_duplicates(inplace=True)

# Hay 1 valor nulo en el campo 'quote', lo eliminamos

quotes_cleaned.dropna(subset=['quote'], inplace=True)

# Verificamos nuevamente nulos y duplicados
nulls, duplicates = check_nulls_and_duplicates(quotes_cleaned)
print(f"Después de eliminar duplicados, hay {nulls.sum()} nulos en total\n")
print(f"Después de eliminar duplicados, hay {duplicates} filas duplicadas\n")

Después de eliminar duplicados, hay 0 nulos en total

Después de eliminar duplicados, hay 0 filas duplicadas



In [7]:
# Normalizar y limpiar los textos
print("🔮 Normalizando los textos 🔮\n")
quotes_cleaned['quote'] = quotes_cleaned['quote'].apply(clean_text)
print("🆗 Textos normalizados 🆗\n")

🔮 Normalizando los textos 🔮

🆗 Textos normalizados 🆗



In [8]:
# Supongamos que las frases están en la columna "quote"
# Ajusta el nombre si tu columna se llama distinto
max_len = 20

# Filtrar frases por longitud
filtered = quotes_cleaned[quotes_cleaned['quote'].apply(lambda x: 0 < len(x.split()) <= max_len)]

# Si hay más de 300,000 después del filtro, tomar muestra aleatoria
target_size = 300000
if len(filtered) > target_size:
    reduced_df = filtered.sample(n=target_size, random_state=42).reset_index(drop=True)
else:
    reduced_df = filtered.reset_index(drop=True)

reduced_dataset = reduced_df['quote'].tolist()

In [9]:
reduced_dataset[:5]

['you know youre in love when you cant fall asleep because reality is finally better than your dreams',
 'a friend is someone who knows all about you and still loves you',
 'darkness cannot drive out darkness only light can do that hate cannot drive out hate only love can do that',
 'we accept the love we think we deserve',
 'it is better to be hated for what you are than to be loved for what you are not']

### Creamos los tokens

In [10]:
tokenizer = Tokenizer(oov_token="<UNK>", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(reduced_dataset)

min_freq = 5
filtered_words = {word: count for word, count in tokenizer.word_counts.items() if count >= min_freq}

In [11]:
tokenizer.word_index = {word: idx + 1 for idx, word in enumerate(filtered_words)}  # +1 por el padding
tokenizer.word_index["<UNK>"] = len(tokenizer.word_index) + 1
tokenizer.index_word = {idx: word for word, idx in tokenizer.word_index.items()}
vocab_size = len(tokenizer.word_index) + 1  # +1 por <PAD>

In [12]:
sequences = tokenizer.texts_to_sequences(reduced_dataset)
padded = pad_sequences(sequences, maxlen=20, padding='post', truncating='post')

In [13]:
# Train/test split
X_train, X_test = train_test_split(padded, test_size=0.2, random_state=42)

y_train = np.expand_dims(X_train, -1)
y_test = np.expand_dims(X_test, -1)

y_train = y_train.squeeze(-1)  # Ahora será (num_samples, max_len)
y_test = y_test.squeeze(-1)    # idem para validación


In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((171908, 20), (42978, 20), (171908, 20), (42978, 20))

In [15]:
max_len = X_train.shape[1]

# Preparar y_train desplazado (target = siguiente token)
y_train_shifted = np.zeros_like(X_train)
y_train_shifted[:, :-1] = X_train[:, 1:]
y_train_shifted[:, -1] = 0  # padding al final

y_test_shifted = np.zeros_like(X_test)
y_test_shifted[:, :-1] = X_test[:, 1:]
y_test_shifted[:, -1] = 0


In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, TimeDistributed
from tensorflow.keras.models import Model

# Asegurar vocab_size
vocab_size = max(X_train.max(), y_train_shifted.max()) + 1

# Modelo simple seq2seq
inputs = Input(shape=(max_len,))
x = Embedding(input_dim=vocab_size, output_dim=32)(inputs)
x = GRU(64, return_sequences=True)(x)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [17]:
from tensorflow.keras.callbacks import Callback

# Recuperado de StackOverflow: https://stackoverflow.com/questions/53500047/stop-training-in-keras-when-accuracy-is-already-1-0

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, threshold=0.6):
        super().__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get("accuracy")
        if acc is not None and acc >= self.threshold:
            print(f"\n✅ Accuracy alcanzó {acc:.2%}, deteniendo entrenamiento.")
            self.model.stop_training = True

In [18]:
### Esto se demora 3 minutos a 38 segundos

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=1000)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)

stop_callback = TerminateOnBaseline(threshold=0.60)

model.fit(train_ds, epochs=5, callbacks=[stop_callback])



Epoch 1/5
10742/10745 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6919 - loss: 2.3520
✅ Accuracy alcanzó 85.20%, deteniendo entrenamiento.
10745/10745 ━━━━━━━━━━━━━━━━━━━━ 198s 18ms/step - accuracy: 0.6920 - loss: 2.3516


In [19]:
# Este proceso se demora casi 3 minutos
import numpy as np

batch_size = 64
predictions = []

for i in range(0, len(X_test), batch_size):
    x_batch = X_test[i:i+batch_size]
    y_batch_pred = model.predict(x_batch, verbose=0)

    # Tomar solo los índices de clase más probables
    y_batch_class = np.argmax(y_batch_pred, axis=-1)

    predictions.append(y_batch_class)

# Unir todos los batches predichos en un solo array
y_pred = np.vstack(predictions)


### Exactitud: Determine la cantidad de aciertos que tiene su modelo por nivel de token por posición.

In [20]:
def frequency_accuracy(y_true, y_pred):
    total = 0
    correct = 0
    for true_seq, pred_seq in zip(y_true, y_pred):
        true_counts = Counter(true_seq)
        pred_counts = Counter(pred_seq)
        matches = sum(min(true_counts[token], pred_counts[token]) for token in true_counts)
        total += len(true_seq)
        correct += matches
    return 100 * correct / total

def accuracy(y_true, y_pred):
    # Calcular la precisión
    correct = np.sum(y_true == y_pred)
    total = np.prod(y_true.shape)
    return correct / total


In [21]:
# Convertir a tokens predichos por posición
y_pred_tokens = np.argmax(y_pred, axis=-1)  # (batch, max_len)

freq_acc = frequency_accuracy(y_test, y_pred)
test_accuracy = accuracy(y_test, y_pred)


In [22]:
print(f"El accuracy por la cantidad de aciertos que tiene su modelo\npor nivel de token por posición es del {test_accuracy*100:.2f}%")

El accuracy por la cantidad de aciertos que tiene su modelo
por nivel de token por posición es del 97.39%


#### También calcule la cantidad de aciertos por frecuencia de palabras entre la frase original y la frase generada, sin importar el lugar en el que esté un token.

In [23]:
print(f"El accuracy por la cantidad de aciertos que tiene su modelo\npor nivel de token independiente de la posición es del {freq_acc:.2f}%")

El accuracy por la cantidad de aciertos que tiene su modelo
por nivel de token independiente de la posición es del 97.39%


Comente sobre sus resultados y qué pueden indicar los valores obtenidos sobre cómo el modelo
predice las palabras. (extensión máxima 3-4 lı́neas)

Errores de palabras: Comente sobre cuáles son las palabras que más errores presenta su modelo, y
comente a qué podrı́a deberse esto (frecuencia, ambigüedad, falta de datos, etc.). (extensión máxima
5 lı́neas)

In [24]:
from collections import Counter

def errores_por_token(y_true, y_pred):
    errores = Counter()
    for true_seq, pred_seq in zip(y_true, y_pred):
        for t, p in zip(true_seq, pred_seq):
            if t != p:
                errores[t] += 1
    return errores.most_common()


In [37]:
errores = errores_por_token(y_test, y_pred)
top_ten = [(tokenizer.index_word.get(int(token_id), "[UNK]"), count) for token_id, count in errores[:10]]


In [41]:
from tabulate import tabulate

tabla = [(i+1, palabra, count) for i, (palabra, count) in enumerate(top_ten)]

# Mostrar tabla
print(tabulate(tabla, headers=["N°", "Palabra", "Errores"], tablefmt="fancy_grid", colalign=("center", "center", "center")))

╒══════╤═════════════╤═══════════╕
│  N°  │   Palabra   │  Errores  │
╞══════╪═════════════╪═══════════╡
│  1   │  immortal   │    24     │
├──────┼─────────────┼───────────┤
│  2   │   status    │    22     │
├──────┼─────────────┼───────────┤
│  3   │   refuses   │    20     │
├──────┼─────────────┼───────────┤
│  4   │    tends    │    18     │
├──────┼─────────────┼───────────┤
│  5   │    ball     │    18     │
├──────┼─────────────┼───────────┤
│  6   │ challenged  │    17     │
├──────┼─────────────┼───────────┤
│  7   │     due     │    16     │
├──────┼─────────────┼───────────┤
│  8   │   shield    │    16     │
├──────┼─────────────┼───────────┤
│  9   │   paradox   │    16     │
├──────┼─────────────┼───────────┤
│  10  │ immediately │    15     │
╘══════╧═════════════╧═══════════╛


Segun el [Corpus of Contemporary American English](https://www.english-corpora.org/coca/)

* immortal está en la posición
\# 9885

* status está en la posición
\# 1274
* refuse está en la posición
\# 1339
* tends está en la posición
\# 1182
* ball está en la posición
\# 877
* challenged está en la posición
\# 26810
* due está en la posición
\# 1277
* shield está en la posición
\# 4562
* paradox está en la posición
\# 7447
* immediately está en la posición
\# 1210

Con eto puedo concluir, que tiene sentido que no haya identificado bien estas palbras, ya que no son tan comunes.


Compresor: Comente si este tipo de modelos es bueno para realizar compresión de texto. Con base
en lo que investigo y determinó con sus resultados, decida si servirı́a o no para este propósito, ya
sea con mejores ajustes, mayor capacidad de cómputo, entrenamiento, más memoria, etc. (extensión
máxima 5-7 lı́neas)
Seq2Seq: Investigue sobre las redes Seq2Seq, mencione diferencias y similitudes sobre el modelo
que usted desarrolló y este tipo de redes. (extensión máxima 5-6 lı́neas)